In [21]:
import glob, os
import librosa
from pyAudioAnalysis import audioBasicIO as aIO
from pyAudioAnalysis import audioSegmentation as aS
import scipy.io.wavfile as wavfile
import random

In [28]:
# base_path = "/home/bikram/Speech/DriData/data/"
# raw_path = base_path + "raw/"

base_path = "/home/deepak/Comp/asr_data_preprocessing_v2/"
raw_path = base_path + "Full_Data/en-in/"


# dataset = "Matsen/"
dataset = "train/"

base_path = "/home/deepak/Comp/asr_data_preprocessing_v2/"

processed_path = base_path + 'test_jugal_out/'

dataset_path = base_path + "RAW/"  #raw_path+datasetdataset_path = base_path + ""  #"RAW/"  #raw_path+dataset

In [29]:
def filtered_files(files_path):
    os.chdir(files_path)
    count = 0
    in_files = []
    out_files = []
    for file in glob.glob(files_path + "*" + ".wav"): #glob.glob("*/*.wav"):
        #print(file, os.path.getsize(file))
        #All valid wav files
        if os.path.getsize(file) >10:
            if librosa.get_duration(filename=file) > 3.0:
                #Filter all conversation where talk time is greater that 90 second
                #print(file, librosa.get_duration(filename=file))
                count = count + 1
                if file.endswith('.wav'):
                    in_files.append(file)
                else:
                    out_files.append(files_path + '/' + file)
    print("count",count, len(in_files), len(out_files))
    return in_files, out_files

In [30]:
in_files, out_files = filtered_files(dataset_path)

count 1 1 0


In [31]:
in_files[0]
len(in_files)

1

In [32]:
def audio_segments(input_list):
    merged_segments = []
    
    #Filter all list items which have audio duration less than 1 second
    list_segments = list(filter(lambda x: x[1]-x[0] > 1.0, input_list))
    
    print(list(list_segments))

    segment_count = len(list_segments)
    start_idx = 0
    #print("All_Segments", segment_count)
    
    while start_idx < segment_count:
        print("Start-", start_idx)
        n=0
        total_duration = 0
        audio_duration = 0
        
        while list_segments[n+start_idx][1] - list_segments[start_idx][0] < 11.0:

            print("Val", list_segments[n+start_idx][1] - list_segments[start_idx][0])

            #early stop if we cross the file duration
            if n + 1 + start_idx >= segment_count:
                total_duration = -1
                #print("break", n + start_idx)
                break
            else:
                n = n + 1

        if total_duration < 0:
            #print("Exit the file end")
            break

            
        total_duration = list_segments[n + start_idx][1] - list_segments[start_idx][0]
                
        index = start_idx
        while index <= n + start_idx:
            audio_duration = audio_duration + list_segments[index][1] - list_segments[index][0]
            index += 1
        
        if audio_duration < 2 or total_duration > 29.5 or total_duration < 10.5:
            start_idx = start_idx + 1
            print("Audio duration low - increment start index")
            continue
            
            
        merged_segments.append([list_segments[start_idx][0],list_segments[start_idx+n][1]])
        
        #print("Iter-", start_idx, n, list_segments[start_idx:n + start_idx +1])
        #print("Dur-", total_duration, audio_duration,[list_segments[start_idx][0],list_segments[start_idx+n][1]])
        
        #if n ==0:
        #    print("Current segment greater than 11", list_segments[n+start_idx][1] - list_segments[start_idx][0])
        
        start_idx = start_idx + n + 1
        
    return merged_segments
    

In [33]:
def segment_file(file, outpath, only_audio = True, write_segment = True):
    
    filename = os.path.basename(file)
    
    [Fs, x] = aIO.read_audio_file(file)
    print(Fs,x)
    segments = aS.silence_removal(x, Fs, 0.05, 0.05, smooth_window = 1.0, weight = 0.3, plot = False)
    
    
    filter_segments = []
    if (only_audio == True):
        #Only take segment greater than 3 seconds
        filter_segments = list(filter(lambda x: x[1]-x[0] > 3.0 and x[1]-x[0] < 20.0, segments))
    else:
        filter_segments = audio_segments(segments)
       
    print(filename, filename[0:-4])
    
    if write_segment:
        for i, s in enumerate(filter_segments):
            print('Start:',s[0],s[1])
            start_buffer = 2.0
            end_buffer = 2.0
            
            if (s[0] > 2.0) :
                #Silence prefix between  1 to 2 second
                random_silence_prefix = random.random()
                start_buffer = s[0] - start_buffer + random_silence_prefix
            else:
                start_buffer = s[0]
            
            end_buffer =s[1] + end_buffer
            
            
            strOut = "{0:}___{1:}_{2:.2f}-{3:.2f}_{4:.2f}.wav".format(filename[0:-4], i, start_buffer, end_buffer, s[1]-s[0])
            print(strOut)
            try:
                wavfile.write(outpath +  "/" + strOut, Fs, x[int(Fs * start_buffer):int(Fs * end_buffer)])
            except:
                print('Failed', strOut)
    
    return filter_segments
    

In [34]:
# fp = "/home/bikram/Speech/DriData/data/raw/Matsen/04/918585010196_DMI011@MattsenKumar.com_2019-11-26-15-31-06-in.wav"
# segment_file(fp, processed_path, True)

In [35]:
only_audio = True
def segment_files(files, processed_path):
    for file in files:
        print("=============", file)
        segment_file(file, processed_path, only_audio)

segment_files(in_files, processed_path)


============= /home/deepak/Comp/asr_data_preprocessing_v2/RAW/006568270132_6500_2017-11-09-12-19-59-out.wav
8000 [8 8 8 ... 8 8 8]
006568270132_6500_2017-11-09-12-19-59-out.wav 006568270132_6500_2017-11-09-12-19-59-out
Start: 10.75 25.35
006568270132_6500_2017-11-09-12-19-59-out___0_9.18-27.35_14.60.wav
Start: 25.55 42.25
006568270132_6500_2017-11-09-12-19-59-out___1_24.22-44.25_16.70.wav
Start: 46.800000000000004 57.95
006568270132_6500_2017-11-09-12-19-59-out___2_45.61-59.95_11.15.wav
Start: 59.400000000000006 63.35
006568270132_6500_2017-11-09-12-19-59-out___3_57.57-65.35_3.95.wav
Start: 64.4 69.95
006568270132_6500_2017-11-09-12-19-59-out___4_62.76-71.95_5.55.wav
Start: 74.45 85.25
006568270132_6500_2017-11-09-12-19-59-out___5_73.20-87.25_10.80.wav
Start: 90.0 106.30000000000001
006568270132_6500_2017-11-09-12-19-59-out___6_88.26-108.30_16.30.wav
Start: 107.5 125.85000000000001
006568270132_6500_2017-11-09-12-19-59-out___7_106.28-127.85_18.35.wav
Start: 134.45000000000002 142.55
00

In [39]:
print('DONE')

DONE


In [11]:
#segments = segment_file(in_files[0])

In [12]:
#print(segments)

In [195]:
audio_segments([(0, 5), (5,5.5), (5.5, 31)]) #,(20,200)])

[(0, 5), (5.5, 31)]
All_Segments 2
Start- 0 SEGEMENT COUNT ::  2
Start- 1 SEGEMENT COUNT ::  2


[[5.5, 31]]

In [196]:
audio_segments([(0, 4), (4,5.5), (5.5, 28)]) #,(20,200)])

[(0, 4), (4, 5.5), (5.5, 28)]
All_Segments 3
Start- 0 SEGEMENT COUNT ::  3


[[0, 28]]

In [197]:
audio_segments([(0, 12), (12,12.5), (12.5, 50)]) #,(20,200)])

[(0, 12), (12.5, 50)]
All_Segments 2
Start- 0 SEGEMENT COUNT ::  2
Start- 1 SEGEMENT COUNT ::  2


[[0, 12]]

In [198]:
audio_segments([(0, 4), (4,5.5), (5.5, 29)]) #,(20,200)])

[(0, 4), (4, 5.5), (5.5, 29)]
All_Segments 3
Start- 0 SEGEMENT COUNT ::  3


[[0, 29]]

In [201]:
audio_segments([(0, 4),(14,15.5), (15.5, 16), (16, 44)])

[(0, 4), (14, 15.5), (16, 44)]
All_Segments 3
Start- 0 SEGEMENT COUNT ::  3
DUR IS UNDER 11 ==> CURRENT n =  0 CURR START_INDEX ::  0
Val 4 0 DIFF ::  4
n incremented -------- False
CALUCALTING TOTAL DUR when n = 1 and start_idx is  0 IS ::  15.5
INDEX ::  1 STRT_IDX ::  0 True
INDEX ::  2 STRT_IDX ::  0 False
Iter- 0 1 [(0, 4), (14, 15.5)]
Start- 2 SEGEMENT COUNT ::  3
CALUCALTING TOTAL DUR when n = 0 and start_idx is  2 IS ::  28
INDEX ::  3 STRT_IDX ::  2 False
Iter- 2 0 [(16, 44)]


[[0, 15.5], [16, 44]]